# Module Code

In [1]:
import json, glob, boto3, os
import pdb
import pandas as pd
from fastparquet import write, ParquetFile
import pytz
import s3fs
import botocore
from collections import Iterable
from typing import List, Union

/Users/paul/anaconda3/lib/python3.7/site-packages/fastparquet/dataframe.py:5: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import CategoricalIndex, RangeIndex, Index, MultiIndex


In [2]:
session = boto3.Session()
BUCKET_NAME = 'snowbot-pv'

# S3 Connect
s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

In [81]:
# parquet engines don't handle shifted timezones
TZ = pytz.timezone('America/Vancouver')

DATA_DIR = "../data/"
MERGED_JSON_FILENAME = "merged_file.json"
merged_json_file = DATA_DIR + MERGED_JSON_FILENAME

# Used for weather data in jsons_to_df()
weather_meta_fields = [
    'newSnow', 'last24Hours', 'last48Hours', 'last7Days', 'midMountainBase',
    'resortID'
]
weather_record_path = ['weather', 'weatherForecast']
weather_meta = [['weather', i] for i in weather_meta_fields]
weather_meta.append('timestamp')

# Used for lift and terrain status in jsons_to_df()
# Important to set categories because when writing incrementally to parquet, some increments
# may not include all statuses.  Manually setting the categories avoids errors due to
# different catergory indexing between increments.
status_cat_dtype = pd.api.types.CategoricalDtype(categories=['X', 'H', 'O'],
                                                 ordered=True)
groomed_cat_dtype = pd.api.types.CategoricalDtype(categories=['No', 'Yes'],
                                                  ordered=True)

# Column dtypes that are to be set for each dataframe
df_dtypes = {
    "lifts": {
        'liftID': 'category',
        'resortID': 'category',
        'liftName': 'category',
        'status': status_cat_dtype,
        'timeToRide': 'object'
    },
    'terrain': {
        'runID': 'category',
        'resortID': 'category',
        'groomed': groomed_cat_dtype,
        'runName': 'category',
        'runType': 'category',
        'status': status_cat_dtype,
        'terrainName': 'category'
    },
    'weather': {
        'resortID': 'category',
        'forecast.dayDescription': 'object',
        'forecast.daycode': 'category',
        'forecast.forecastString': 'object',
        'forecast.iconName': 'object',
        'forecast.summaryDescription': 'object',
        'forecast.temperatureHigh': 'object',
        'forecast.temperatureLow': 'object',
        'weather.last24Hours': 'object',
        'weather.last48Hours': 'object',
        'weather.last7Days': 'object',
        'weather.midMountainBase': 'object',
        'weather.newSnow': 'object'
    }
}


def flatten(items):
    """Yield items from any nested iterable"""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
            yield x


# The columns that serve to identify records for each topic
topic_ID_col_names = {
    'lifts': ['resortID', 'liftName'],
    'terrain': ['resortID', 'runID', 'runName', 'terrainName'],
    'weather': 'resortID',
    'all_topics': 'timestamp'
}
# All of the column names that serve to identify records in at least one of the topics
all_ID_col_names = set(flatten(topic_ID_col_names.values()))

In [4]:
# from https://alexwlchan.net/2019/07/listing-s3-keys/
def get_matching_s3_objects(bucket, prefix="", suffix=""):
    """
    Generate objects in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    kwargs = {'Bucket': bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix, )
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                return

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(bucket, prefix="", suffix=""):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    for obj in get_matching_s3_objects(bucket, prefix, suffix):
        yield obj["Key"]


def merge_matching_jsons_on_s3(save_file, prefix="", suffix=""):
    """Merges json files on S3 that match the suffix into a new json and save it
    as the save_file on S3."""

    result = []

    for f in get_matching_s3_keys(BUCKET_NAME, prefix=prefix, suffix=suffix):

        # Write the file from S3 into a local temp file
        with open('temp', 'wb') as tfw:
            bucket.download_fileobj(f, tfw)

        # Append the local temp file into the result list
        with open('temp', 'rb') as tfr:
            result.append(json.load(tfr))

    os.remove("temp")

    # Fill the output file with the merged content
    with open(save_file, "w") as outfile:
        json.dump(result, outfile)

# TBD: more efficient to go straight to df w/o saving json to file


def set_df_datatypes(df, topic):
    """Set the datatypes for a df according to the topic that
    it represents."""
    df = df.astype(df_dtypes[topic])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    return df


def jsons_to_df(jsons, record_path, meta='timestamp'):
    """Convert a json containing one or more timestamps to a dataframe."""
    if record_path == 'weather':
        # Deal with the nested object that the weather data uses to store the weather forecast
        df = pd.json_normalize(jsons, record_path=weather_record_path,
                               meta=weather_meta, record_prefix='forecast.')
        df.rename(columns={"weather.resortID": "resortID"}, inplace=True)
    else:
        df = pd.json_normalize(jsons, record_path=record_path,
                               meta=meta)

    df = set_df_datatypes(df, record_path)
    return df


def load_json_as_df(merged_json_file, record_path):
    """Load json file containing one or more timestamps as a dataframe."""
    with open(merged_json_file, "r") as f:
        d = json.load(f)
        df = jsons_to_df(d, record_path)
        return df


def get_data_changes(df, topic, keep_oldest=False):
    """
    Filter out rows that do not represent changed data.

    Parameters
    ----------
    df : pandas.DataFrame
        Includes 'timestamp' identifying and data columns.  Lists data for each timestamp.
    keep_oldest : boolean
        Indicates if the returned DataFrame should keep the oldest record for each entity (i.e.
        lift, resort, tor terrain) even if an entity has no data changes.  This is so that the
        earliest data for each entity is not lost, and all entities are listed the returned DataFrame
        even if their data has not changed.  Use `False` when there is just one DataFrame to process.
        Use `True` is cases where the data changes will be appended to an existing dataframe that
        already has at least one row for each entity.

    Returns
    -------
    pandas.DataFrame
        Only includes the rows from the original dataframe where there was a change to new values
        in the data columns.
    """
    ID_columns = topic_ID_col_names[topic]
    data_columns = [c for c in df.columns if c not in all_ID_col_names]

    def filter_for_data_changes(df, keep_oldest=keep_oldest):
        """Filter out rows where data is unchanged for adjacent timestamps.
        Required to handle cases when there are > 2 rows per entity.
        """
        keep_idx = df[data_columns].ne(df[data_columns].shift()).any(
            axis=1).values[1:]  # True for rows with data changes
        changed_rows = df.reset_index(drop=True).drop(index=0)[keep_idx]

        if keep_oldest:
            firstrow = df.loc[df['timestamp'].idxmin()]
            keep_df = firstrow.to_frame().T.append(changed_rows)
        else:
            keep_df = changed_rows

        return keep_df

    # Drop any rows that are complete duplicates so that conditional evaluation will
    # work.  This is required for Peak 2 Peak Gondola because it is duplicated in the
    # lifts data.  Maybe others as well.
    df.drop_duplicates(inplace=True)

    # 1 means that there were up to 2 rows fond per group
    if df.groupby(ID_columns, group_keys=False).cumcount().max() < 2:
        # Most efficient method.  Only works if there are 2 or less rows per entity.
        subset = df.columns.drop('timestamp')
        df = df.sort_values('timestamp')

        if keep_oldest:
            df = df.drop_duplicates(subset=subset, keep='first')
        else:
            df = df.drop_duplicates(subset=subset, keep=False)
            df = df.drop_duplicates(subset=ID_columns, keep='last')

    else:
        # Less efficient method.  Required if there are > 2 rows per entity.
        df = df.sort_values('timestamp').groupby(ID_columns, group_keys=False)\
               .apply(filter_for_data_changes)\
               .reset_index(drop=True)

    df = set_df_datatypes(df, topic)

    return df

# Process lift json fies

In [19]:
def get_status_durations(lifts_df):
    '''Calculate values and add columns for the time difference between the
    timestamp for the current status and the timestamp for the next status
    for each lift:
    `time_diff` column: Gives the duration that the lift was in the status indicated in the `status` column.
    `time_diff_seconds` column: `time_diff` converted to seconds.
    
    lifts_status_changes_df should be TBD
    '''
    # TBD: optimize if needed via # 3 under:
    # https://towardsdatascience.com/pandas-tips-and-tricks-33bcc8a40bb9
    df = lifts_df.sort_values(by=['resortID', 'liftID', 'timestamp'])
    df['time_diff'] = df.groupby(['resortID', 'liftID'])['timestamp'].diff(1).shift(-1)

    # Fill in the durations which will be missing for the most recent status changes
    missing_time_diffs_idx = df.loc[(df['time_diff'].isnull()) & (
        df['timestamp'] >= df['timestamp'].min()), 'timestamp'].index.values

    df.loc[missing_time_diffs_idx, 'time_diff'] = df['timestamp'].max(
    ) - df.loc[missing_time_diffs_idx, 'timestamp']

    # Convert to seconds
    df['time_diff_seconds'] = df['time_diff'].dt.total_seconds()

    return df

## Whistler Lifts

In [171]:
merge_matching_jsons_on_s3(suffix="lifts.json", save_file=merged_json_file)

In [172]:
whis_lifts_df = load_json_as_df(merged_json_file, 'lifts')

whis_lifts_status_changes_df = get_data_changes(whis_lifts_df, 'lifts', keep_oldest=True)

In [ ]:
lifts_status_changes_df

**NOTE:** `timeToRide` is just the time is takes to ride the lift, not the current wait time:

In [174]:
whis_lifts_df.groupby("liftName")['timeToRide'].unique()

liftName
7th Heaven Express                 [6]
Big Red Express                    [8]
Blackcomb Gondola Lower            [7]
Blackcomb Gondola Upper            [7]
Catskinner Express                 [4]
Coca-Cola Tube Park                [4]
Creekside Gondola                  [7]
Crystal Ridge Express              [7]
Emerald 6 Express                  [6]
Excalibur Gondola Lower            [3]
Excalibur Gondola Upper            [5]
Excelerator Express                [6]
Fitzsimmons Express                [6]
Franz's Chair                      [8]
Garbanzo Express                   [7]
Glacier Express                    [6]
Harmony 6 Express                  [6]
Horstman T-Bar                     [4]
Jersey Cream Express               [5]
Magic Chair                        [6]
Olympic Chair                      [5]
Peak 2 Peak Gondola               [12]
Peak Express                       [3]
Showcase T-Bar                     [3]
Symphony Express                   [7]
T-Bars          

In [175]:
whis_lifts_df

,liftID,resortID,liftName,status,timeToRide,timestamp
0,69,13,Blackcomb Gondola Lower,X,7,2020-01-03 00:19:09.631011-08:00
1,70,13,Blackcomb Gondola Upper,X,7,2020-01-03 00:19:09.631011-08:00
2,5,13,Excalibur Gondola Lower,X,3,2020-01-03 00:19:09.631011-08:00
3,71,13,Excalibur Gondola Upper,X,5,2020-01-03 00:19:09.631011-08:00
4,8,13,Excelerator Express,X,6,2020-01-03 00:19:09.631011-08:00
...,...,...,...,...,...,...
113617,44,13,Franz's Chair,X,8,2020-02-13 20:16:09.008557-08:00
113618,43,13,Peak Express,X,3,2020-02-13 20:16:09.008557-08:00
113619,37,13,Harmony 6 Express,X,6,2020-02-13 20:16:09.008557-08:00
113620,42,13,Symphony Express,X,7,2020-02-13 20:16:09.008557-08:00


In [177]:
whis_lifts_df = get_status_durations(whis_lifts_status_changes_df)

# Uses local date formatting, otherwise Tableau will mix up month and day
# alternatively, can export to json:
# lifts_status_changes_df.to_json(DATA_DIR + "lifts_status_changes.json", orient='table')
whis_lifts_df.to_csv(DATA_DIR + "whis_lifts_status_changes.csv", date_format='%c')


In [197]:
# add:
# 
# daily: for each chair calculate most open status of the day: O > H > X
# Days since each chair was last seen open with timestamp of most recent open time.
# snowfall since last open
# save data for other mountains

### Loop through JSON files for all topics

In [23]:
for topic in ['lifts', 'terrain', 'weather']:
    merge_matching_jsons_on_s3(suffix=topic + ".json", save_file=merged_json_file)
    df = load_json_as_df(merged_json_file, topic)
    status_changes_df = get_data_changes(df, topic, keep_oldest=True)
    
    if topic == 'lifts':
        get_status_durations(status_changes_df).to_csv(DATA_DIR + 'whis_lifts_status_changes.csv', date_format='%c')
    else:
        status_changes_df.to_csv(DATA_DIR + 'whis_' + topic + '_status_changes.csv', date_format='%c')

In [185]:
%pastebin [-d “Custom description”] 1-7

'http://dpaste.com/3QAKY1G'

# Storage options testing

In [ ]:
df.to_pickle(DATA_DIR + "df_test.pkl")

In [ ]:
from fastparquet import write

# parquet engines don't handle shifted timezones
import pytz
TZ = pytz.timezone('America/Vancouver')
df['timestamp'] = df.timestamp.dt.tz_convert(pytz.utc)

In [ ]:
# Note: May need snappy-python as a req to run on AWS Lambda
df.to_parquet(DATA_DIR + "df_test.parquet", engine='fastparquet')

In [ ]:
load_df = pd.read_parquet(DATA_DIR + "df_test.parquet")
load_df['timestamp'] = load_df.timestamp.dt.tz_convert(TZ) # convert back to correct timezone


In [ ]:
#TBD convert back to correct datatypes
load_df.dtypes

In [ ]:
df.to_csv(DATA_DIR + "df_test.csv")

Test file size results:
- json: 800 Kb?
- csv: 474 Kb
- pickle: 145 Kb
- parquet: 15 Kb

## Delta Lake Notes

Requires apache spark instance.  For future use, could set one up to work with lambda using https://aws.amazon.com/emr/features/spark/?

Otherwise databricks (similar to QxMD project)

# Parquet on S3

For all topics from the EpicMix API.  Compare most recent topic data from json on S3 and if the data has changes, append the changes to parquet file on S3.

### Module Code

In [5]:
import requests
from datetime import datetime
fs = s3fs.S3FileSystem()
myopen = fs.open
nop = lambda *args, **kwargs: None

HISTORY_SUFFIX = '_history_DEV.parquet'
PRIOR_SUFFIX = '_prior_DEV.json'


def write_dataframe_to_parquet_on_s3(df, topic, fname):
    """ Write a dataframe to a Parquet file on S3.  Creates a new parquet file if one
    doesn't already exist.
    """

    def write_parquet(df, fname, app=True):

        output_file = f"s3://{BUCKET_NAME}/{fname}"
        write(output_file,
              df,
              # partition_on=['timestamp'],
              file_scheme='hive',
              append=app,  # need to remove or catch exception to work when file doesn't exist
              open_with=myopen,
              mkdirs=nop)
        print(f"Writing {len(df)} records to {fname}.")

    # Unshift the timezone because parquet engines don't handle shifted timezones
    df.loc[:, 'timestamp'] = df.loc[:, 'timestamp'].dt.tz_convert(pytz.utc)

    s3_object = bucket.Object(fname)

    if not list(bucket.objects.filter(Prefix=fname)):
        print(f"File {fname} not found.  Creating new file.")
        # Keep oldest record for each entity because creating new file
        df = get_data_changes(df, topic=topic, keep_oldest=True)
        write_parquet(df, fname, app=False)

    else:
        print(f"File {fname} found on S3.")
        df = get_data_changes(df, topic=topic, keep_oldest=False)
        write_parquet(df, fname, app=True)


def filter_resort(data, resortID: int = None) -> dict:
    """Filter for a specific resort."""
    if resortID:
        return data["resortID"] == resortID
    else:
        return data


def get_data(filter_topic: Union[str, List] = None, filter_resortID: int = None) -> dict:
    """Get data from EpicMix API. Defaults to all resorts.  Option to filter for a
    specific resort
    """
    API_URL = 'http://www.epicmix.com/vailresorts/sites/epicmix/api/mobile/'
    # keys are used in the requests, the values and used in the response
    DATA_LIST = {'lifts': 'lifts',
                 'weather': 'snowconditions', 'terrain': 'terrains'}
    json_data = dict()

    # Create lists to filter by topic
    if filter_topic is not None:
        filtered_data_list = {k: v for k,
                              v in DATA_LIST.items() if k in filter_topic}
    else:
        filtered_data_list = DATA_LIST

    for d, name in filtered_data_list.items():
        res = requests.get(API_URL + d + '.ashx')
        res.raise_for_status()
        data = json.loads(res.text)[name]
        data = list(filter(lambda x: filter_resort(x, filter_resortID), data))
        json_data[d] = json.dumps(
            {'timestamp': str(datetime.now(TZ)), d: data})

    return json_data


def s3_object_exists(fname):
    """Check if an s3 object exists.  Returns `True` if the object exists."""
    try:
        bucket.Object(fname)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print(f"{fname} doesn't exist")
        else:
            raise
    return True


def load_dataframe_from_parquet_on_s3(fname):
    """ Load a dataframe from a Parquet file on S3. """
    if s3_object_exists(fname):
        read_file = f"s3://{BUCKET_NAME}/{fname}"
        pf = ParquetFile(read_file, open_with=myopen)
        df = pf.to_pandas()

        # Reshift the timezone because parquet engines don't handle shifted timezones
        df.loc[:, 'timestamp'] = df.loc[:, 'timestamp'].dt.tz_convert(TZ)

        return df


class api_data():
    def __init__(self, topic: str, current_json: str):
        self.topic = topic
        self.current_json = current_json
        # May not exist yet
        self.prior_fname = topic + PRIOR_SUFFIX
        self.prior_object = bucket.Object(self.prior_fname)
        self.check_prior_object()

    def check_prior_object(self):
        """Get prior data json"""
        try:
            self.prior_object.load()
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print(f"Prior json for {self.topic} doesn't exist")
                self.prior_exists = False
            else:
                # Something else has gone wrong.
                raise
        else:
            self.prior_exists = True
        return self.prior_exists

    def get_prior_data_json(self):
        """Get prior data json from S3."""
        if self.prior_exists == True:
            prior = self.prior_object.get()['Body'].read().decode('utf-8')
            self.prior_json = json.loads(prior)
            print(f"Loaded prior {self.topic} json data from S3")
            return self.prior_json
        else:
            print(f"Prior json for {self.topic} doesn't exist")

    def data_changed(self):
        """Compare current data json with prior data json without their timestamps.  The timestamps
        on the current json will always be more recent even when none of the other data has changed.
        """
        if self.prior_json[self.topic] == self.current_json[self.topic]:
            print(
                f"No differences between current and prior {self.topic} data were found.")
            return False
        else:
            print(
                f"Found differences between current and prior {self.topic} data.")
            return True

    def save_prior_data(self):
        """Save the current data as prior data on S3."""
        bucket.put_object(Key=self.prior_fname,
                          Body=bytes(json.dumps(self.current_json).encode('UTF-8')))


class ParquetWriter():
    """Identifies new data and writes it to Parquet file on S3."""

    def __init__(self):
        # Get current data
        self.data_current_all = get_data()  # String.

    def write_new_data_all(self):
        """Writes new data for each type (i.e. 'lift', 'weather', 'terrian')
        of data returned by the API.
        """
        for topic in self.data_current_all:
            current_json = json.loads(self.data_current_all[topic])
            data = api_data(topic, current_json)
            self.write_new_data(data)

    def write_new_data(self, api_data):
        """If current data has changed since the last update of Parquet file is, add it
        to the Parquet file.  Save the current data as json to serve as the prior for
        the next comparison.
        """

        if api_data.prior_exists:
            api_data.get_prior_data_json()
            if api_data.data_changed():
                # Get a df with the chages between the prior and current json data
                df = jsons_to_df(
                    [api_data.prior_json, api_data.current_json], record_path=api_data.topic)
                write_dataframe_to_parquet_on_s3(
                    df, api_data.topic, api_data.topic + HISTORY_SUFFIX)

                # save current data json as prior
                api_data.save_prior_data()
                print(
                    f"Replaced data in {api_data.prior_object.key} with current data.")
        else:
            print(f"Prior json for {api_data.topic} doesn't exist")
            # Create the prior file
            api_data.save_prior_data()
            print(f"Created {api_data.prior_fname}")
        print('\n')

In [53]:
%%time
pr = ParquetWriter()
pr.write_new_data_all()

Loaded prior lifts json data from S3
Found differences between current and prior lifts data.
File lifts_history_DEV.parquet found on S3.
Writing 7 records to lifts_history_DEV.parquet.
Replaced data in lifts_prior_DEV.json with current data.


Loaded prior weather json data from S3
Found differences between current and prior weather data.
File weather_history_DEV.parquet found on S3.
Writing 14 records to weather_history_DEV.parquet.
Replaced data in weather_prior_DEV.json with current data.


Loaded prior terrain json data from S3
Found differences between current and prior terrain data.
File terrain_history_DEV.parquet not found.  Creating new file.
Writing 1987 records to terrain_history_DEV.parquet.
Replaced data in terrain_prior_DEV.json with current data.


CPU times: user 484 ms, sys: 47 ms, total: 531 ms
Wall time: 3.64 s


In [24]:
import time
while True:
    print('\n\n\n' + time.ctime() + ':\n---------------------')
    pr = ParquetWriter()
    pr.write_new_data_all()
    time.sleep(3600)




Thu Feb 13 13:06:29 2020:
---------------------
Loaded prior lifts json data from S3
No differences between current and prior lifts data were found.


Loaded prior weather json data from S3
Found differences between current and prior weather data.
File weather_history_DEV.parquet found on S3.
Writing 11 records to weather_history_DEV.parquet.
Replaced data in weather_prior_DEV.json with current data.


Loaded prior terrain json data from S3
Found differences between current and prior terrain data.
File terrain_history_DEV.parquet found on S3.
Writing 18 records to terrain_history_DEV.parquet.
Replaced data in terrain_prior_DEV.json with current data.





Thu Feb 13 14:06:33 2020:
---------------------
Loaded prior lifts json data from S3
Found differences between current and prior lifts data.
File lifts_history_DEV.parquet found on S3.
Writing 14 records to lifts_history_DEV.parquet.
Replaced data in lifts_prior_DEV.json with current data.


Loaded prior weather json data from S3
F

KeyboardInterrupt: 

**Warnings**

See https://github.com/dask/fastparquet/issues/477 for fastparquet warnings about `RangeIndex._start, RangeIndex._stop, RangeIndex._step`


    /Users/paul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
    /Users/paul/anaconda3/lib/python3.7/site-packages/fastparquet/writer.py:655: FutureWarning: RangeIndex._start is deprecated and will be removed in a future version. Use RangeIndex.start instead
      index_cols = [{'name': index_cols.name, 'start': index_cols._start,
    /Users/paul/anaconda3/lib/python3.7/site-packages/fastparquet/writer.py:656: FutureWarning: RangeIndex._stop is deprecated and will be removed in a future version. Use RangeIndex.stop instead
      'stop': index_cols._stop, 'step': index_cols._step,
    /Users/paul/anaconda3/lib/python3.7/site-packages/fastparquet/writer.py:656: FutureWarning: RangeIndex._step is deprecated and will be removed in a future version. Use RangeIndex.step instead
      'stop': index_cols._stop, 'step': index_cols._step,

### Load parquet and save as .csv

In [24]:
parq_df = load_dataframe_from_parquet_on_s3('lifts' + HISTORY_SUFFIX)
lifts_status_changes_parq_df = get_status_durations(parq_df)
lifts_status_changes_parq_df.to_csv(
    DATA_DIR + "lifts_status_changes_parq.csv", date_format='%c')

### Testing

In [12]:
TEST_DATA_DIR = "../data/test/"
TEST_VALIDATION_DATA_DIR = TEST_DATA_DIR + "valid/"
lifts_json_test_file = TEST_DATA_DIR + "lifts_test.json"
terrain_json_test_file = TEST_DATA_DIR + "terrain_test.json"
weather_json_test_file = TEST_DATA_DIR + "weather_test.json"
merged_lifts_json_test_file = TEST_DATA_DIR + "merged_lifts_test.json"
merged_terrain_json_test_file = TEST_DATA_DIR + "merged_terrain_test.json"
merged_weather_json_test_file = TEST_DATA_DIR + "merged_weather_test.json"
merged_whis_lifts_json_test_file = TEST_DATA_DIR + "merged_whis_lifts_test.json"

#### Find column combinations to identify entities

In [13]:
from itertools import combinations

# Move to EDA notebook
# Do we really need to use all the terrain columns in topic_ID_col_names to uniquely identify each run?
ID_cols = topic_ID_col_names['terrain']
df = load_json_as_df(terrain_json_test_file, 'terrain')

for combo in combinations(ID_cols, len(ID_cols)-1):
    print(f"Combo: {combo}\tDuplicates: {df.duplicated(combo).sum()}")

Combo: ('resortID', 'runID', 'runName')	Duplicates: 1
Combo: ('resortID', 'runID', 'terrainName')	Duplicates: 0
Combo: ('resortID', 'runName', 'terrainName')	Duplicates: 2
Combo: ('runID', 'runName', 'terrainName')	Duplicates: 0


We will always get one duplicate lift entry because the Peak 2 Peak Gondola is returned twice in the API data:

In [14]:
ID_cols = ['resortID', 'liftName', 'liftID']
df = load_json_as_df(lifts_json_test_file, 'lifts')

for combo in combinations(ID_cols, len(ID_cols)-1):
    print(f"Combo: {combo}\tDuplicates: {df.duplicated(combo).sum()}")

print(f"Combo: {ID_cols}\tDuplicates: {df.duplicated(ID_cols).sum()}")

Combo: ('resortID', 'liftName')	Duplicates: 1
Combo: ('resortID', 'liftID')	Duplicates: 2
Combo: ('liftName', 'liftID')	Duplicates: 1
Combo: ['resortID', 'liftName', 'liftID']	Duplicates: 1


#### Tests

In [15]:
import unittest

In [25]:
# new


class TestNotebook(unittest.TestCase):

    def test_get_data(self):
        self.assertEqual(
            list(get_data(filter_topic=['lifts']).keys()), ['lifts'],
            'Returned dictionary was not filtered for the right topic'
        )
        self.assertEqual(
            list(get_data(filter_topic=['lifts', 'terrain']).keys()),
            ['lifts', 'terrain'],
            'Returned dictionary was not filtered for the right topics'
        )
        self.assertEqual(
            set(get_data().keys()), {'lifts', 'weather', 'terrain'},
            'Returned dictionary was not filtered for all topics'
        )

    def test_get_data_changes(self):

        tests_df = pd.DataFrame({
            'test_file': [merged_whis_lifts_json_test_file,
                          merged_lifts_json_test_file,
                          merged_terrain_json_test_file,
                          merged_weather_json_test_file],
            'topic': ['lifts', 'lifts', 'terrain', 'weather'],
            'validation_fname_prefix': ['get_data_changes_merged_whis_lifts',
                                        'get_data_changes_merged_lifts',
                                        'get_data_changes_merged_terrain',
                                        'get_data_changes_merged_weather']
        })

        for row in tests_df.iterrows():

            test_file = row[1]['test_file']
            df = load_json_as_df(test_file, row[1]['topic'])
            df = df.sample(frac=1)  # Shuffle the data

            for keep_oldest in [True, False]:
                if keep_oldest == True:
                    validation_fname_suffix = '_keep_oldest_valid.json'
                else:
                    validation_fname_suffix = '_drop_oldest_valid.json'

                tested_df = get_data_changes(
                    df, row[1]['topic'], keep_oldest=keep_oldest)

                valid_file = TEST_VALIDATION_DATA_DIR + \
                    row[1]['validation_fname_prefix'] + validation_fname_suffix
                valid_df = pd.read_pickle(valid_file)

                #  Sort and reindex before comparison because we are not testing the indexes
                # or row orders that the functions return.
                tested_df.sort_values(
                    tested_df.columns.to_list(), ignore_index=True, inplace=True)
                valid_df.sort_values(
                    valid_df.columns.to_list(), ignore_index=True, inplace=True)

                pd.testing.assert_frame_equal(
                    tested_df, valid_df,
                    f"Result from {test_file} did not match validation dataframe {valid_file}."
                )


    def test_terrain_ID_col_names(self):
        df = load_json_as_df(terrain_json_test_file, 'terrain')
        ID_cols = topic_ID_col_names['terrain']
        self.assertEqual(
            df.duplicated(ID_cols).sum(), 0,
            f"{ID_cols} are not sufficient to uniquely identify the terrain records."
        )

    # Assert df has no NaN or NaTs:
    # assert parq_df.isnull().sum().sum() == 0


unittest.main(argv=[''], verbosity=2, exit=False)

test_get_data (__main__.TestNotebook) ... ok
test_get_data_changes (__main__.TestNotebook) ... ok
test_terrain_ID_col_names (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 3.213s

OK


In [166]:
# test that the status categories are complete for lifts and terrain
# maybe like this or direct access the index
test_terrain = load_dataframe_from_parquet_on_s3(topic + HISTORY_SUFFIX)
test_terrain.status.unique()

[O, X]
Categories (2, object): [O, X]

In [37]:
# TBD: test dtypes if not covered by type hint testing
jsons_to_df(lifts_prior_json, 'lifts').dtypes

liftID                                      category
resortID                                    category
liftName                                    category
status                                      category
timeToRide                                    object
timestamp     datetime64[ns, pytz.FixedOffset(-480)]
dtype: object

In [152]:
topic = 'terrain'

parq_df = load_dataframe_from_parquet_on_s3(topic + HISTORY_SUFFIX)
parq_df.dtypes

runID                                   category
resortID                                category
groomed                                 category
runName                                 category
runType                                 category
status                                  category
terrainName                             category
timestamp      datetime64[ns, America/Vancouver]
dtype: object

In [153]:
parq_df = set_df_datatypes(parq_df, topic)
parq_df.dtypes

runID                                   category
resortID                                category
groomed                                 category
runName                                 category
runType                                 category
status                                  category
terrainName                             category
timestamp      datetime64[ns, America/Vancouver]
dtype: object

In [155]:
sort = ['timestamp']
sort.extend(topic_ID_col_names[topic])
print(f"sorted by {sort}")
parq_df.sort_values(sort)

sorted by['timestamp', 'resortID', 'runID', 'runName', 'terrainName']


,runID,resortID,groomed,runName,runType,status,terrainName,timestamp
index,,,,,,,,
44,10,1,No,38,3,O,Vail Village,2020-02-13 15:06:38.957648-08:00
127,10,1,No,Apres Vous,3,O,Back Bowls,2020-02-13 15:06:38.957648-08:00
87,10,1,No,Baccarat,2,O,Lionshead,2020-02-13 15:06:38.957648-08:00
171,10,1,Yes,Big Rock Park,2,O,Blue Sky Basin,2020-02-13 15:06:38.957648-08:00
0,10,1,No,Blue Ox,3,O,Golden Peak,2020-02-13 15:06:38.957648-08:00
...,...,...,...,...,...,...,...,...
3372,1333,13,No,Where's Joe,3,O,Excelerator – Blackcomb Gondola Upper,2020-02-13 19:30:15.445546-08:00
3373,1334,13,Yes,X Course,3,O,Excelerator – Blackcomb Gondola Upper,2020-02-13 19:30:15.445546-08:00
3374,1335,13,No,Yard Sale,3,O,Excelerator – Blackcomb Gondola Upper,2020-02-13 19:30:15.445546-08:00


In [156]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(parq_df.sort_values(sort).query('resortID == 13'))

,runID,resortID,groomed,runName,runType,status,terrainName,timestamp
index,,,,,,,,
1426,130,13,Yes,7th Avenue,1,O,7th Heaven,2020-02-13 15:06:38.957648-08:00
1347,130,13,Yes,Adagio - Lower,2,O,Symphony Amphitheatre,2020-02-13 15:06:38.957648-08:00
1243,130,13,Yes,Adult Learning/ Supercarpet,1,O,Village - Olympic - Fitzsimmons,2020-02-13 15:06:38.957648-08:00
1441,130,13,No,Arthur's Choice,3,O,Crystal Zone,2020-02-13 15:06:38.957648-08:00
1301,130,13,No,Back Bowl,3,O,Harmony,2020-02-13 15:06:38.957648-08:00
1285,130,13,Yes,Banana Peel,2,O,Big Red - Franz's - Garbanzo,2020-02-13 15:06:38.957648-08:00
1376,130,13,No,Bark Sandwich,3,O,Excelerator – Blackcomb Gondola Upper,2020-02-13 15:06:38.957648-08:00
1320,130,13,No,Big Timber,3,O,The Peak - T Bar,2020-02-13 15:06:38.957648-08:00
1457,130,13,No,Blackcomb Glacier,3,O,Glacier,2020-02-13 15:06:38.957648-08:00


In [96]:
# Filter dataframe by date
parq_df = parq_df[parq_df['timestamp'] > '2020-02-08']

In [161]:
parq_df_dup = parq_df[parq_df.duplicated(subset=sort, keep=False)]

In [162]:
parq_df_dup

,runID,resortID,groomed,runName,runType,status,terrainName,timestamp
index,,,,,,,,


For testing terrain:
resorts:21, UD: 2, Name "Bear Paw" . Terrain: "Arrowhead" and "Resort Skiways (Access to/from Homesites and Lodging)" 

In [163]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(parq_df_dup.sort_values(['resortID', 'runID', 'runName', 'timestamp']))

,runID,resortID,groomed,runName,runType,status,terrainName,timestamp
index,,,,,,,,


We can uniquely identify each run via a combination of: `resortID`, `runID` and `runName`:

In [165]:
topic_ID_col_names[topic]

['resortID', 'runID', 'runName', 'terrainName']

In [164]:
assert(
    parq_df.drop_duplicates(subset=['resortID', 'runID', 'runName'], keep=False).equals(
    parq_df.drop_duplicates(subset=['resortID', 'runID', 'runName', 'terrainName'], keep=False))
)

AssertionError: 

TBD: testing
- result is expected size
- no duplicate rows
- no duplicate information in adjacent rows by time

### S3 Object Deletion

In [277]:
# Delete prior jsons
bucket.Object('lifts' + PRIOR_SUFFIX).delete()
bucket.Object('terrain' + PRIOR_SUFFIX).delete()
bucket.Object('weather' + PRIOR_SUFFIX).delete()

{'ResponseMetadata': {'RequestId': '431A1A9902A35164',
  'HostId': 'vBxSvKa39QA+h00WLr3Y+pTQ/H6D+rejlCt9SBaqonAAT+8vvXyJoQcAGYfnTiU62xskiTzBQpE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'vBxSvKa39QA+h00WLr3Y+pTQ/H6D+rejlCt9SBaqonAAT+8vvXyJoQcAGYfnTiU62xskiTzBQpE=',
   'x-amz-request-id': '431A1A9902A35164',
   'date': 'Mon, 10 Feb 2020 19:19:15 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [278]:
# Delete parquet history files
bucket.objects.filter(Prefix='lifts' + HISTORY_SUFFIX + '/').delete()
bucket.objects.filter(Prefix='terrain' + HISTORY_SUFFIX + '/').delete()
bucket.objects.filter(Prefix='weather' + HISTORY_SUFFIX + '/').delete()

[]

In [168]:
# Load live whistler parquet data

%time parq_df = load_dataframe_from_parquet_on_s3(HISTORY_FNAME)

lifts_status_changes_parq_df = get_status_durations(parq_df)
lifts_status_changes_parq_df.to_csv(DATA_DIR + "lifts_status_changes_parq.csv", date_format='%c')
lifts_status_changes_parq_df

CPU times: user 1.21 s, sys: 51.9 ms, total: 1.26 s
Wall time: 11.4 s


,liftID,resortID,liftName,status,timeToRide,timestamp,time_diff,time_diff_seconds
0,3,13,7th Heaven Express,O,6,2020-02-03 14:21:57.064742-08:00,00:47:48.055160,2868.055160
29,3,13,7th Heaven Express,X,6,2020-02-03 15:09:45.119902-08:00,18:11:29.035149,65489.035149
67,3,13,7th Heaven Express,O,6,2020-02-04 09:21:14.155051-08:00,05:49:50.203591,20990.203591
79,3,13,7th Heaven Express,X,6,2020-02-04 15:11:04.358642-08:00,18:39:59.624647,67199.624647
140,3,13,7th Heaven Express,O,6,2020-02-05 09:51:03.983289-08:00,05:19:59.834930,19199.834930
...,...,...,...,...,...,...,...,...
622,72,13,Whistler Village Gondola Upper,O,11,2020-02-12 08:21:04.096543-08:00,07:19:59.792787,26399.792787
657,72,13,Whistler Village Gondola Upper,X,11,2020-02-12 15:41:03.889330-08:00,15:59:59.968475,57599.968475
669,72,13,Whistler Village Gondola Upper,H,11,2020-02-13 07:41:03.857805-08:00,00:40:00.298287,2400.298287
691,72,13,Whistler Village Gondola Upper,O,11,2020-02-13 08:21:04.156092-08:00,07:19:59.808515,26399.808515


In [124]:
parq_df.status.cat.categories

Index(['X', 'H', 'O'], dtype='object')

In [125]:
print(*parq_df.status)

O O O O O O O O O O O O X X O O O X O O O X O O O X O O O X X X X X X X X X X X X X X X X X X X X O O O O O O O O O O O O O O O O O O O O O O O O O X O


In [186]:
parq_df.sort_values(["liftName", "timestamp"])

,liftID,resortID,liftName,status,timeToRide,timestamp
0,3,13,7th Heaven Express,O,6,2020-02-03 14:21:57.064742-08:00
29,3,13,7th Heaven Express,X,6,2020-02-03 15:09:45.119902-08:00
67,3,13,7th Heaven Express,O,6,2020-02-04 09:21:14.155051-08:00
1,36,13,Big Red Express,O,8,2020-02-03 14:21:57.064742-08:00
35,36,13,Big Red Express,X,8,2020-02-03 15:37:30.556825-08:00
...,...,...,...,...,...,...
47,33,13,Whistler Village Gondola Lower,X,5,2020-02-03 15:37:30.556825-08:00
65,33,13,Whistler Village Gondola Lower,O,5,2020-02-04 09:11:14.369598-08:00
28,72,13,Whistler Village Gondola Upper,O,11,2020-02-03 14:21:57.064742-08:00
48,72,13,Whistler Village Gondola Upper,X,11,2020-02-03 15:37:30.556825-08:00


### Issue when running get_status_durations(parq_df)
Resulting in error:

    ~/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py in from_codes(cls, codes, categories, ordered, dtype)
        705 
        706         if len(codes) and (codes.max() >= len(dtype.categories) or codes.min() < -1):
    --> 707             raise ValueError("codes need to be between -1 and " "len(categories)-1")
        708 
        709         return cls(codes, dtype=dtype, fastpath=True)

    ValueError: codes need to be between -1 and len(categories)-1


Same error seen when running `parq_df[['status']].sort_values(by=['status'])`

This was caused by missing categories (`H`) in the `status` column (and maybe others)

#### Code to inspect issue:

In [228]:
# Test for issue
parq_df[['status']].sort_values(by=['status'])

,status
0,H
59,H
58,H
57,H
56,H
...,...
38,O
39,O
40,O
42,O


In [280]:
parq_df.status.cat.categories

Index(['X', 'H', 'O'], dtype='object')

In [281]:
print(*parq_df.status.cat.codes)

2 2 2 2 2 0 2 2 2 2 2 2 2 2 0 2 2 2 0 2 2 2 0 2 0 2 0 2 2 2


In [205]:
len(parq_df.status.cat.codes)

65

In [185]:
# Should be false
parq_df.status.cat.codes.max() >= len(parq_df.status.dtype.categories)

False

In [186]:
# Should be false
parq_df.liftName.cat.codes.min() < -1

False

In [ ]:
for c in parq_df.columns:
    print(parq_df[c].cat.categories)

In [36]:
parq_df.dtypes

liftID                   category
resortID                 category
liftName                 category
status                   category
timeToRide                  int64
timestamp     datetime64[ns, UTC]
dtype: object

In [113]:
lifts_status_changes_df.dtypes

liftID                                      category
resortID                                    category
liftName                                    category
status                                      category
timeToRide                                     int64
timestamp     datetime64[ns, pytz.FixedOffset(-480)]
dtype: object

In [106]:
parq_df["timestamp"] = pd.to_datetime(pd.Series(np.asarray(parq_df["timestamp"])))

In [73]:
read_file = f"s3://{BUCKET_NAME}/{fname}.parquet"
pf = ParquetFile(read_file, open_with=myopen)

# Check the categories for a specific row group
pf.grab_cats(columns='status', row_group_index=1)

{'status': array(['H', 'O', 'X'], dtype=object)}

In [66]:
# If partitioning by column, gives known values for each column
pf.cats

{}

#### Possible solutions
1. Remove partitioning by date column when writing to parquet
**2. Set status categories manually via `set_categories`. (and any other columns with the same issue.  See https://github.com/dask/dask/issues/2944**
3. Leave problem columns as text-based when writing and loading from parquet

# Testing timestamps for file loading

In [51]:
read_file = f"s3://{BUCKET_NAME}/{fname}.parquet"
pf = ParquetFile(read_file, open_with=myopen)
test = pf.to_pandas()["timestamp"]

NameError: name 'fname' is not defined

In [52]:
# If needed: to convert for categorical datetime to regular datetime
df["timestamp"] = pd.to_datetime(pd.Series(np.asarray(df["timestamp"])))

In [53]:
test.dt = test.dt.tz_convert(tz= 'America/Vancouver')

NameError: name 'test' is not defined

/Users/paul/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:597: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.


more info: https://pandas-docs.github.io/pandas-docs-travis/whatsnew/v0.24.0.html#converting-timezone-aware-series-and-index-to-numpy-arrays

In [54]:
load_dataframe_from_parquet_on_s3(fname).dtypes

NameError: name 'fname' is not defined

### Testing local parquet saves

In [ ]:
def save_parquet(df, fname):
    # parquet engines don't handle shifted timezones
    df.loc[:, 'timestamp'] = df.loc[:, 'timestamp'].dt.tz_convert(pytz.utc)

    # Note: May need snappy-python as a req to run on AWS Lambda
    df.to_parquet(DATA_DIR + fname + '.parquet',
                  engine='fastparquet',
                  partition_on=['timestamp'],
                  file_scheme='mixed')

In [91]:
save_parquet(df[0:3].copy(), 'wb_lifts_history')

In [92]:
df.iloc[20:22, :].copy().to_parquet(DATA_DIR + 'wb_lifts_history' + '.parquet',
              engine='fastparquet',
              partition_on=['timestamp'],
              file_scheme='mixed',
              append=True)
# Catch exception that is doesn't exist here

In [ ]:
# todo: change time_diff to "duration"
# test on lambda
# make datatype dict for and general set datatypes function


# Utilities

In [32]:
def load_prior_json_from_s3(topic: str) -> dict:
    """E.g. load_prior_json_from_s3('weather')"""
    prior_object = bucket.Object(topic + PRIOR_SUFFIX)
    prior = prior_object.get()['Body'].read().decode('utf-8')
    return json.loads(prior)

In [33]:
# TBD: Add to scraping 
# check that weather_prior_json['weather'][0].keys() matches list of expected columns (in case new ones are in use)
# Add as exception handling for the other topics as well
load_prior_json_from_s3('weather')['weather'][0].keys()

dict_keys(['newSnow', 'last24Hours', 'last48Hours', 'last7Days', 'midMountainBase', 'resortID', 'weatherForecast'])

In [36]:
# load a json from S3
terrain_prior_json = load_prior_json_from_s3('terrain')

# Convert json to a dataframe normally wer can use jsons_to_df()
terrain_prior_df = pd.json_normalize(
    data=terrain_prior_json,
    record_path=['terrain'],
    meta='timestamp'
)

terrain_prior_df.columns

Index(['runID', 'resortID', 'groomed', 'runName', 'runType', 'status',
       'terrainName', 'timestamp'],
      dtype='object')

In [118]:
def del_lifts_history():
    bucket.Object(HISTORY_FNAME).delete()

def del_lifts_prior():
    bucket.Object(PRIOR_STATUS_FNAME).delete()

In [89]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(df2,
                              indicator=True,
                              how='outer')
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    return diff_df



,liftID,resortID,liftName,status,timeToRide,timestamp,_merge


In [121]:
del_lifts_history()
del_lifts_prior()

# Notes
- Terrain data: runs need to be identified via combination of `resortID`, `runName` and `runID`
- There are run IDs that repeat for the same resort (e.g. for Vail resortID == 1, runID == 10
- TBD: Are the combination of `resortID`, `runID`, and `runType` always unique?

## To do
- differentiate among the 3 diff pieces of API data by 'subject'
- option to get data for specific subject(s)
- return data object with filter_by_resort() method
- live_data class?  Subclass for each subject?